---
title: Centum Getting Start
subtitle: Water Accounting usnig Centum
license: CC-BY-4.0
github: https://github.com/executablebooks/mystjs
subject: Tutorial
authors:
  - name: Benjamin Mary
    email: benjamin.mary@ica.csic.es
    corresponding: true
    orcid: 
    affiliations:
      - ICA-CSIC
date: 2025/01/10
---

## Welcome to using Centum 

:::{important} Installing Centum
```bash
pip install centum
```
:::

In [2]:
#!pip install centum

In [13]:
!python ../setup.py install

457.21s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


No parent package detected, impossible to derive `name`
running install
/home/z0272571a@CAMPUS.CSIC.ES/miniconda3/envs/Bousval/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/home/z0272571a@CAMPUS.CSIC.ES/miniconda3/envs/Bousval/lib/python3.9/site-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running

In [14]:
from centum import plotting as pltC
import numpy as np
import xarray as xr

from centum.irrigation_district import IrrigationDistrict 



In [ ]:
ETAnalysisConfig.

In [15]:


from centum.delineation import ETAnalysis
import matplotlib.pyplot as plt

from centum.irrigation_district import IrrigationDistrict 
from centum import irrigation_district as irr_geo_tools 


ImportError: cannot import name 'ETAnalysis' from 'centum.delineation' (/home/z0272571a@CAMPUS.CSIC.ES/miniconda3/envs/Bousval/lib/python3.9/site-packages/centum/delineation.py)